# Use Studio notebook lifecycle configurations

This notebook implements a method for configuring dependencies within SageMaker Studio by using Studio Lifecycle Configuration whereby dependencies are installed each time a kernel starts-up.

## Configure environment

Edit "Add-Domain-ID" to be your SageMaker Domain Id. 

In [ ]:
import base64
import boto3
import sagemaker

SageMakerDomainId='<Add-Domain-ID>'

client = boto3.client('sagemaker')

Create Studio Lifecyle Configuration

In [ ]:
StudioLifecycleConfigContent = """# This script installs a single pip package on a SageMaker Studio Kernel Application
#!/bin/bash
set -eux
# PARAMETERS
PACKAGE=pyarrow
pip install --upgrade $PACKAGE"""

message_bytes = StudioLifecycleConfigContent.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)

response = client.create_studio_lifecycle_config(
    StudioLifecycleConfigName='InstallDependencies',
    StudioLifecycleConfigContent=base64_bytes.decode('ascii'),
    StudioLifecycleConfigAppType='KernelGateway',
)

Add Studio Lifecyle Configuration to the SageMaker domain

In [ ]:
client.update_domain(
    DomainId=SageMakerDomainId,
    DefaultUserSettings={
        'KernelGatewayAppSettings': {
            'LifecycleConfigArns': [
                response['StudioLifecycleConfigArn'],
            ],
        }
    }
)

Now that all the configuration is done, it is time to test the script within Studio. To do this, launch the Studio and on the Launcher tab, locate the Notebooks and compute resources section and click on the Change environment to select the lifecycle configuration you created.

![](../img/studio-create-notebook.png)

On the Change environment popup, change the Start-up script option to be the lifecycle configuration created in previous steps.

![](../img/change-environment.png)

Back in the Launcher window, click on the Create notebook option.
You can also set the Lifecycle configuration to be run by default in the Lifecycle configurations for personal Studio apps section of the Domain page.
Within the new notebook, the dependencies installed in the start-up script will be available.

![](../img/lifecycle-config-working.png)